# Модель ARIMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

Спецификация ARIMA(p,d,q) (ARIMA = AutoRegressive Integrated Moving Average)

$$
\begin{aligned}
	\Delta ^d y_t &= \left(\sum_{j=0}^m\alpha_j t^j\right)+\sum_{j=1}^p\phi_j\Delta^d y_{t-j}+u_t+\sum_{s=1}^q\theta_s u_{t-s} &
	u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где
* p - порядок авторегрессионной части
* d - порядок интегрирования
* q - порядок части скользящего среднего 
* $\alpha_0$ - константа (d=0) или снос (d=1)

Спецификация с использование лагового оператора

$$
\begin{aligned}
	\Phi(L)(1-L)^dy_t&=\left(\sum_{j=0}^k\alpha_j t^j\right)+\Theta(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где многочлены
$$
\begin{aligned}
	\Phi(z)&=1-\phi_1z-\cdots-\phi_pz^p & \Theta(z)&=1+\theta_1z+\cdots+\theta_qz^q
\end{aligned}
$$

Альтернативная спецификация (оцениваемая в `statsmodels`!)

$$
\begin{aligned}
	y_t&=\sum_{j=d}^{d+m}\delta_jt^j+\epsilon_t & \Phi(L)(1-L)^d\epsilon_t&=\Theta(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$

In [ ]:
import numpy as np
import pandas as pd

# импорт из библиотеки statsmodels
from sktime.forecasting.arima import StatsModelsARIMA as ARIMA
# импорт из пакета pmdarima
from sktime.forecasting.arima import AutoARIMA

from sktime.utils.plotting import plot_series
# временной горизонт для прогнозирования
from sktime.forecasting.base import ForecastingHorizon

import pandas_datareader.data as web

# тесты диагностики
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Для подгонки используем класс 
* [StatsModelsARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.StatsModelsARIMA.html): модель заданного порядка (из пакета `statsmodels`)
* [StatsForecastAutoARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.statsforecast.StatsForecastAutoARIMA.html): выбор модели "оптимального порядка" (из пакета `statsforecast`)

При спецификации модели заданного порядка нужно задать параметры `order` (порядок модели) и  `trend` (порядок тренда, если тренд включается в модель)

|d|Порядок тренда $m+d$|`trend`|`trend` (альтернативно)|
|-|:-:|:-:|:-:|
|0|-|'n'|`[0]`|
|0|0|'c'|`[1]`|
|0|1|'ct'|`[1, 1]`|
|1|-|'n'|`[0]`|
|1|1|'t'|`[0, 1]`|

Альтернативно можно использовать классы (фактически из пакета `pmdarima`)
* [ARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.ARIMA.html): модель заданного порядка
* [AutoARIMA](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.arima.AutoARIMA.html): выбор модели "оптимального порядка"


## Подгонка ARIMA заданного порядка

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARIMA(2,1,2) без сноса (без тренда) для `y`

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=(1+\theta_1  L+\theta_2 L^2) u_{t}
$$

In [ ]:
forecaster = ARIMA(order=(2,1,2), trend='n')
forecaster.fit(y)
forecaster.summary()

In [ ]:
forecaster.get_fitted_params()

## Диагностика модели

Остатки (__обратим внимание, что нужно удалить первые $d$ значений!__)

In [ ]:
# Порядок дифференцирования
d = forecaster.get_params()['order'][1]
# остатки модели
resids = forecaster.predict_residuals(y)[d:]
resids

График остатков

In [ ]:
resids.plot()

plt.show()

Тест на серийную корреляцию (Ljung-Box)

In [ ]:
# model_df = p+q
acorr_ljungbox(resids, lags=[7], model_df=2+2)

Тест на гетероскедастичность

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(resids, nlags=7, ddof=2+2)

lm_stat, lm_pval

## Прогнозирование

Построим прогноз  по модели ARIMA(1,1,1) со сносом

Спецификация

$$
	(1-\phi L)(1-L) y_t=\alpha_0+(1+\theta L) u_{t}
$$

Численный прогноз на 10 периодов

In [ ]:
forecaster = ARIMA(order=(1,1,1), trend='t')
# зададим горизонт прогнозирования и частотность
fh = ForecastingHorizon(np.arange(1,11), freq ='W-Fri')

y_pred = forecaster.fit_predict(y=y, fh=fh)
y_pred

Визуализация прогноза 

In [ ]:
plot_series(y.tail(50), y_pred, labels=['y', 'y_pred'])

plt.show()

Доверительный интервал

In [ ]:
conf_int = forecaster.predict_interval(fh=fh, coverage=0.9)
conf_int

In [ ]:
plot_series(y.tail(20), y_pred, labels=['y', 'y_pred'], markers=['o', 'X'], pred_interval=conf_int)

plt.show()

# Автоматический выбор порядка модели

Базовые параметры автоматического выбора

|Критерий|`AutoARIMA`|Значения|
|-|-|-|
|Информационный критерий|`information_criterion`|aic (по умолчанию), aicc, bic, hqic |
|тест единичного корня|`test`|kpss (по умолчанию), adf, pp|
|max порядок d|`max_d`| 2 по умолчанию |
|max порядок p,q|`max_p`, ` max_q`|5 по умолчанию|
|модель с сезонностью|`seasonal`|True по умолчанию|
|уровень значимость|`alpha`|0.05 по умолчанию|

__Замечание__: автоматические выбирается нужно ли включать снос/константу/тренд в модель

In [ ]:
forecaster = AutoARIMA(information_criterion='bic', test='kpss', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']